1. Instalando e importando as bibliotecas necessárias: pandas e pandera para a extração e validação dos dados, mysql connector para criar a base de dados local e realizar a carga dos dados. Mantenha comentado caso já tenha as bibliotecas instaladas.

In [1]:
#pip install pandas


In [2]:
#pip install mysql-connector-python

In [3]:
#pip install pandera

In [4]:
import pandas as pd
import pandera as pa

2. Extraindo arquivo CSV com os dados e convertendo os dados para os tipos corretos. A opção do tratamento e limpeza dos dados aqui foi de não excluir nenhuma linha apesar dos erros e valores ausentes, por se tratar de uma tabela com a finalidade de coletar valores monetários. Dessa forma, operações de contabilidade seriam alteradas pela exclusão arbitrária desses valores. A decisão de excluir ou não essas linhas do banco de dados é pertinente aos analistas que conhecem a necessidade do negócio.

In [5]:
df = pd.read_csv("base_vendas_2020.csv", sep=";", parse_dates=['data_compra_date'], infer_datetime_format=True, dayfirst=True) #importa a tabela

In [6]:
df.replace(['', ' ', 'NULL'], pd.NA, inplace=True) #padroniza valores nulos em todas as colunas

In [7]:
df = df.where((pd.notnull(df)), None) #linha de código adicionada porque alguns valores nulos eram lidos como NaN e causavam erro na etapa de carga na base de dados 

In [8]:
df.fillna(value={'id_produto':0}, inplace=True) #tratamento adicional para a coluna de id_produto para garantir a inserção de dados. Há linhas na tabela com valores ausentes nessa coluna.

In [9]:
#substituindo as vírgulas por ponto nas colunas de receita e convertendo os tipos dos valores para float 
df['receita_frete'] = df['receita_frete'].str.replace(',','.').astype(float)
df['receita_produto'] = df['receita_produto'].str.replace(',','.').astype(float)
df['desconto_produto'] = df['desconto_produto'].str.replace(',','.').astype(float)
df['receita_total'] = df['receita_total'].str.replace(',','.').astype(float)

In [10]:
df['unique_id'] = pd.RangeIndex(stop=df.shape[0]) #cria uma chave substituta (surrogate key) para servir de chave primária no banco de dados, uma vez que na coluna id_pedido há valores duplicados para os casos de vendas com mais de um produto.

3. Criando schema para validar os dados

In [11]:

schema = pa.DataFrameSchema(
    {
        "data_compra_date": pa.Column(pa.DateTime),
        "id_pedido": pa.Column(pa.Int),      
        "id_produto": pa.Column(pa.Int, required=False, nullable=True),
        "entrega_uf": pa.Column(pa.String),
        "receita_frete": pa.Column(pa.Float),
        "receita_produto": pa.Column(pa.Float),
        "quantidade_produto": pa.Column(pa.Int),
        "situacao": pa.Column(pa.String),
        "tipo_venda": pa.Column(pa.String),
        "canal": pa.Column(pa.String),
        "desconto_produto": pa.Column(pa.Float, nullable=True),
        "receita_total": pa.Column(pa.Float),
        "nome_produto": pa.Column(pa.String, required=False, nullable=True),
        "categoria": pa.Column(pa.String, required=False, nullable=True),
        "unique_id": pa.Column(pa.Int)
        
        
    },
    
    strict=True,
    coerce=True,
)

In [12]:
schema.validate(df).head(5)

,data_compra_date,id_pedido,id_produto,entrega_uf,receita_frete,receita_produto,quantidade_produto,situacao,tipo_venda,canal,desconto_produto,receita_total,nome_produto,categoria,unique_id
0,2020-01-30,7830233,544113,PR,36.860001,180.899994,1,Aprovado,1P,MM,-14.17,203.589994,Escrivaninha Cavalete 4 Prateleiras Hawaii Yes...,Escrivaninhas,0
1,2020-01-06,7521840,148376,SP,29.059999,469.899994,1,Aprovado,1P,B2W,-46.99,451.969992,Conjunto Sala de Jantar Mesa Tampo de Vidro e ...,Conjuntos de Mesas e Cadeiras de Jantar,1
2,2020-01-29,7824799,280052,SP,86.089996,399.899994,1,Aprovado,1P,Carrefour,0.00,485.989990,Armário de Cozinha 12 Portas 1 Gaveta Clara Po...,Armários de Cozinha,2
3,2020-01-23,7761090,553063,MG,108.980003,959.900024,1,Cancelado,1P,MM,0.00,1068.880028,Penteadeira Camarim Retrô com LED 5 Gavetas Ne...,Penteadeiras,3
4,2020-01-14,7644963,1175243,SP,16.080000,142.000000,1,Aprovado,3P,MM,0.00,158.080000,Quadro Abstrato Decorativo - 57x120cm,Quadros,4


4. Conectando com base de dados - Não esqueça de alterar os dados de conexão no código para os de seu banco local!


In [13]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='host', user='user',  
                        password='passwordstring')#não se esqueça de inserir os dados de conexão do seu próprio banco de dados MySQL entre as aspas
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("DROP DATABASE IF EXISTS vendadb;")
        cursor.execute("CREATE DATABASE vendadb")
        print("Base de dados criada")
except Error as e:
    print("Erro ao conectar com MySQL", e)

Base de dados criada


5. Inserindo os dados no BD - Não esqueça de alterar os dados de conexão no código para os de seu banco local!

In [14]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', database='vendadb', user='root', password='bWdJ-W9!NFhB')#não se esqueça de repetir os dados de conexão do seu próprio banco de dados MySQL entre as aspas
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Você se conectou à base de dados: ", record)
        #criando a tabela
        cursor.execute('DROP TABLE IF EXISTS vendas_tb;')
        cursor.execute("CREATE TABLE IF NOT EXISTS vendas_tb(`data_compra` DATE NOT NULL,`idvendas` INT NOT NULL, `id_produto` INT NULL,  `entrega_uf` VARCHAR(2) NULL, `receita_frete` FLOAT NULL, `receita_produto` FLOAT NOT NULL, `qtd_produto` INT NULL, `situacao` VARCHAR(45) NULL, `tipo_venda` VARCHAR(2) NULL, `canal_venda` VARCHAR(45) NULL, `desconto_produto` FLOAT NULL, `receita_total` FLOAT NOT NULL, `nome_produto` VARCHAR(300) NULL, `categoria` VARCHAR(100) NULL, `unique_id` INT NOT NULL, PRIMARY KEY (`unique_id`))")
        print("Tabela criada")
        #inserindo os dados do arquivo na tabela
        for i,row in df.iterrows():
            sql = "INSERT INTO vendadb.vendas_tb VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))            
            conn.commit()
        print("Dados inseridos com sucesso")
except Error as e:
            print("Erro ao conectar ao MySQL: ", e)

Você se conectou à base de dados:  ('vendadb',)
Tabela criada
Dados inseridos com sucesso


6. Testando algumas queries no BD

In [15]:
sql = "SELECT * FROM vendadb.vendas_tb LIMIT 10"
cursor.execute(sql)

result = cursor.fetchall()
for i in result:
    print(i)

(datetime.date(2020, 1, 30), 7830233, 544113, 'PR', 36.86, 180.9, 1, 'Aprovado', '1P', 'MM', -14.17, 203.59, 'Escrivaninha Cavalete 4 Prateleiras Hawaii Yescasa', 'Escrivaninhas', 0)
(datetime.date(2020, 1, 6), 7521840, 148376, 'SP', 29.06, 469.9, 1, 'Aprovado', '1P', 'B2W', -46.99, 451.97, 'Conjunto Sala de Jantar Mesa Tampo de Vidro e 4 Cadeiras Espresso Móveis', 'Conjuntos de Mesas e Cadeiras de Jantar', 1)
(datetime.date(2020, 1, 29), 7824799, 280052, 'SP', 86.09, 399.9, 1, 'Aprovado', '1P', 'Carrefour', 0.0, 485.99, 'Armário de Cozinha 12 Portas 1 Gaveta Clara Poliman Móveis', 'Armários de Cozinha', 2)
(datetime.date(2020, 1, 23), 7761090, 553063, 'MG', 108.98, 959.9, 1, 'Cancelado', '1P', 'MM', 0.0, 1068.88, 'Penteadeira Camarim Retrô com LED 5 Gavetas Nevada Potente', 'Penteadeiras', 3)
(datetime.date(2020, 1, 14), 7644963, 1175243, 'SP', 16.08, 142.0, 1, 'Aprovado', '3P', 'MM', 0.0, 158.08, 'Quadro Abstrato Decorativo - 57x120cm', 'Quadros', 4)
(datetime.date(2020, 1, 17), 7683

In [16]:
sql = "SELECT idvendas, receita_total, entrega_uf FROM vendadb.vendas_tb WHERE entrega_uf='SP' AND situacao='Aprovado' LIMIT 5"
cursor.execute(sql)

result = cursor.fetchall()
for i in result:
    print(i)

(7521840, 451.97, 'SP')
(7824799, 485.99, 'SP')
(7644963, 158.08, 'SP')
(7683578, 144.96, 'SP')
(7495461, 878.9, 'SP')


In [17]:
sql = "SELECT idvendas, receita_total, nome_produto, canal_venda FROM vendadb.vendas_tb WHERE nome_produto LIKE 'aparador%' LIMIT 5"
cursor.execute(sql)

result = cursor.fetchall()
for i in result:
    print(i)

(7695977, 267.66, 'Aparador com Prateleira Creta Artely', 'MM')
(7586764, 286.08, 'Aparador Bar com Adega Sonatta - Off White', 'MM')
(7545624, 321.15, 'Aparador Buffet Retrô Quartzo 3 Portas e Prateleira Ajustável Móveis Bechara', 'B2W')
(7639185, 229.24, 'Aparador Dunas Artely', 'Mercado Livre')
(7823752, 454.11, 'Aparador Buffet Retrô 3 Portas Wood Prime - Preto Fosco - Rp', 'MM')
